# The Logistic Map: Your First Strange Attractor

Welcome to chaos! The logistic map is the simplest equation that exhibits chaotic behavior:

$$x_{n+1} = r \cdot x_n \cdot (1 - x_n)$$

Despite its simplicity, this equation models population growth and shows how chaos emerges from deterministic systems.

In [ ]:
# Import our tools
import numpy as np
import matplotlib.pyplot as plt
from ipywidgets import interact, FloatSlider
import sys
sys.path.append('..')

from src.maps import LogisticMap
from src.visualise import plot_time_series, plot_bifurcation, plot_return_map

# Set up nice plotting
plt.style.use('seaborn-v0_8-darkgrid')
%matplotlib inline

## 1. Understanding the Equation

The logistic map models population growth with limited resources:
- `x` represents population (normalized between 0 and 1)
- `r` is the growth rate parameter
- The term `x_n` represents growth proportional to current population
- The term `(1 - x_n)` represents limitation due to finite resources

Let's see it in action!

In [ ]:
# Create a logistic map with growth rate r=2.5
logistic = LogisticMap(r=2.5)

# Start with initial population x0 = 0.2
trajectory = logistic.iterate(x0=0.2, n=50)

# Visualize the population over time
fig = plot_time_series(trajectory, 
                       title="Logistic Map: Convergence to Fixed Point (r=2.5)",
                       xlabel="Generation", 
                       ylabel="Population")
plt.show()

print(f"Population stabilizes at: {trajectory[-1]:.4f}")

## 2. Interactive Exploration: Changing the Growth Rate

As we increase the growth rate `r`, the system exhibits increasingly complex behavior:
- **r < 1**: Population dies out
- **1 < r < 3**: Population stabilizes at a fixed point
- **3 < r < 3.57**: Period-doubling cascade begins
- **r > 3.57**: Chaos emerges!

In [ ]:
def explore_logistic_map(r=2.5, x0=0.2, n_iterations=100):
    """Interactive exploration of the logistic map."""
    logistic = LogisticMap(r=r)
    trajectory = logistic.iterate(x0=x0, n=n_iterations)
    
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 5))
    
    # Time series
    ax1.plot(trajectory, 'b-', linewidth=0.8)
    ax1.set_title(f'Time Series (r={r:.2f})', fontsize=14)
    ax1.set_xlabel('Iteration')
    ax1.set_ylabel('Population x')
    ax1.set_ylim(-0.1, 1.1)
    ax1.grid(True, alpha=0.3)
    
    # Return map
    ax2.scatter(trajectory[:-1], trajectory[1:], alpha=0.6, s=20)
    ax2.plot([0, 1], [0, 1], 'r--', alpha=0.5, label='y=x')
    
    # Plot the logistic function
    x_vals = np.linspace(0, 1, 100)
    y_vals = r * x_vals * (1 - x_vals)
    ax2.plot(x_vals, y_vals, 'g-', linewidth=2, label=f'f(x) at r={r:.2f}')
    
    ax2.set_title('Return Map', fontsize=14)
    ax2.set_xlabel('$x_n$')
    ax2.set_ylabel('$x_{n+1}$')
    ax2.set_xlim(-0.1, 1.1)
    ax2.set_ylim(-0.1, 1.1)
    ax2.legend()
    ax2.grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()
    
    # Analyze the behavior
    last_values = trajectory[-20:]
    unique_values = len(np.unique(np.round(last_values, 6)))
    
    if unique_values == 1:
        print(f"Fixed point: Population converges to {last_values[-1]:.6f}")
    elif unique_values <= 8:
        print(f"Period-{unique_values} cycle detected")
    else:
        print("Chaotic behavior!")

# Create interactive widget
interact(explore_logistic_map,
         r=FloatSlider(min=0.5, max=4.0, step=0.01, value=2.5, description='Growth rate r:'),
         x0=FloatSlider(min=0.01, max=0.99, step=0.01, value=0.2, description='Initial x€:'),
         n_iterations=(50, 500, 50));

## 3. The Bifurcation Diagram: Route to Chaos

The bifurcation diagram shows how the long-term behavior changes as we vary the parameter `r`. This reveals the famous "period-doubling route to chaos."

In [ ]:
# Generate bifurcation diagram
print("Generating bifurcation diagram... (this may take a moment)")
logistic = LogisticMap()
r_values, x_values = logistic.bifurcation_data(r_range=(2.4, 4.0), n_r=2000)

fig = plot_bifurcation(r_values, x_values,
                       title="Bifurcation Diagram: The Route to Chaos")

# Add annotations for key regions
ax = plt.gca()
ax.axvline(x=3.0, color='red', linestyle='--', alpha=0.5)
ax.text(3.0, 0.9, 'Period-2 begins', rotation=90, va='bottom', ha='right', color='red')

ax.axvline(x=3.57, color='orange', linestyle='--', alpha=0.5)
ax.text(3.57, 0.9, 'Chaos begins', rotation=90, va='bottom', ha='right', color='orange')

plt.show()

## 4. Sensitivity to Initial Conditions

A hallmark of chaos is extreme sensitivity to initial conditions - the "butterfly effect." Let's see how tiny differences in starting values lead to completely different outcomes.

In [ ]:
# Compare trajectories with slightly different initial conditions
r_chaos = 3.8  # Chaotic regime
logistic_chaos = LogisticMap(r=r_chaos)

# Two very close initial conditions
x0_1 = 0.2000
x0_2 = 0.2001  # Just 0.05% different!

traj1 = logistic_chaos.iterate(x0_1, n=100)
traj2 = logistic_chaos.iterate(x0_2, n=100)

# Plot the trajectories
fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(12, 8), sharex=True)

# Individual trajectories
ax1.plot(traj1, 'b-', linewidth=1, label=f'x€ = {x0_1}')
ax1.plot(traj2, 'r-', linewidth=1, label=f'x€ = {x0_2}')
ax1.set_ylabel('Population x')
ax1.set_title(f'Butterfly Effect in the Logistic Map (r={r_chaos})', fontsize=14)
ax1.legend()
ax1.grid(True, alpha=0.3)

# Difference between trajectories
difference = np.abs(traj1 - traj2)
ax2.semilogy(difference, 'g-', linewidth=2)
ax2.set_xlabel('Iteration')
ax2.set_ylabel('|Difference|')
ax2.set_title('Exponential Divergence of Initially Close Trajectories', fontsize=14)
ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print(f"Initial difference: {abs(x0_1 - x0_2):.6f}")
print(f"Final difference: {difference[-1]:.6f}")
print(f"Amplification factor: {difference[-1]/abs(x0_1 - x0_2):.0f}x")

## 5. Finding Order in Chaos: The Feigenbaum Constant

Even in chaos, there's hidden order! The period-doubling cascade follows a universal pattern discovered by Mitchell Feigenbaum.

In [ ]:
# Calculate bifurcation points
def find_period_doubling_points():
    """Find the r values where period doubling occurs."""
    bifurcation_points = [
        3.0,          # Period 1 ’ 2
        3.449490,     # Period 2 ’ 4  
        3.544090,     # Period 4 ’ 8
        3.564407,     # Period 8 ’ 16
        3.568759,     # Period 16 ’ 32
    ]
    return bifurcation_points

# Calculate Feigenbaum constant
points = find_period_doubling_points()
deltas = []

for i in range(len(points) - 2):
    delta = (points[i+1] - points[i]) / (points[i+2] - points[i+1])
    deltas.append(delta)
    print(f"´{i+1} = {delta:.6f}")

print(f"\nFeigenbaum constant ´ H 4.669201...")
print("This constant appears in ALL period-doubling systems!")

# Visualize the period-doubling cascade
fig, ax = plt.subplots(figsize=(10, 6))

for i, r in enumerate(points):
    logistic = LogisticMap(r=r)
    trajectory = logistic.iterate(0.5, n=300)
    # Take last values after transient
    last_values = trajectory[-100:]
    unique = np.unique(np.round(last_values, 6))
    
    ax.scatter([r] * len(unique), unique, c=f'C{i}', s=50, 
               label=f'r={r:.6f} (Period {2**i})')

ax.set_xlabel('Growth rate r', fontsize=12)
ax.set_ylabel('Stable values', fontsize=12)
ax.set_title('Period-Doubling Cascade', fontsize=14)
ax.legend(bbox_to_anchor=(1.05, 1), loc='upper left')
ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## 6. Practical Applications

The logistic map isn't just mathematical curiosity - it models real phenomena:

1. **Population Dynamics**: Animal populations with limited resources
2. **Economics**: Boom-bust cycles in markets
3. **Epidemiology**: Disease spread with saturation effects
4. **Neural Networks**: Activation functions in machine learning

Let's simulate a population over many generations:

In [ ]:
def simulate_population(initial_pop=0.1, growth_rate=2.8, generations=200, 
                       carrying_capacity=1000):
    """Simulate population dynamics with the logistic map."""
    logistic = LogisticMap(r=growth_rate)
    normalized_trajectory = logistic.iterate(initial_pop, n=generations)
    
    # Convert to actual population numbers
    population = normalized_trajectory * carrying_capacity
    
    fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(12, 8))
    
    # Population over time
    ax1.plot(population, 'b-', linewidth=1)
    ax1.axhline(y=carrying_capacity/2, color='r', linestyle='--', 
                alpha=0.5, label='Half capacity')
    ax1.set_ylabel('Population')
    ax1.set_title(f'Population Dynamics (r={growth_rate})', fontsize=14)
    ax1.legend()
    ax1.grid(True, alpha=0.3)
    
    # Phase space (population vs rate of change)
    pop_change = np.diff(population)
    ax2.scatter(population[:-1], pop_change, alpha=0.5, s=10)
    ax2.set_xlabel('Population')
    ax2.set_ylabel('Population Change')
    ax2.set_title('Phase Space: Population vs Change', fontsize=14)
    ax2.grid(True, alpha=0.3)
    ax2.axhline(y=0, color='k', linestyle='-', alpha=0.3)
    
    plt.tight_layout()
    plt.show()
    
    # Statistics
    print(f"Initial population: {int(population[0])}")
    print(f"Mean population: {int(np.mean(population))}")
    print(f"Std deviation: {int(np.std(population))}")
    
    return population

# Interactive population simulator
interact(simulate_population,
         initial_pop=FloatSlider(min=0.01, max=0.5, step=0.01, value=0.1, 
                                description='Initial %:'),
         growth_rate=FloatSlider(min=1.0, max=4.0, step=0.01, value=2.8, 
                                description='Growth rate:'),
         generations=(50, 500, 50),
         carrying_capacity=(100, 10000, 100));

## 7. Exercises and Challenges

### Exercise 1: Fixed Points
Find the mathematical formula for the fixed point of the logistic map when $1 < r < 3$.

*Hint: At a fixed point, $x^* = f(x^*) = r \cdot x^* \cdot (1 - x^*)$*

In [ ]:
# Your solution here
def find_fixed_point(r):
    """Calculate the non-zero fixed point of the logistic map."""
    # At fixed point: x = r*x*(1-x)
    # Solving: x = r*x - r*x^2
    # 1 = r - r*x
    # x* = (r-1)/r
    
    if r <= 1:
        return 0  # Only fixed point is 0
    else:
        return (r - 1) / r

# Test your solution
for r in [2.0, 2.5, 2.9]:
    fixed_point = find_fixed_point(r)
    # Verify it's actually fixed
    logistic = LogisticMap(r=r)
    verified = logistic(fixed_point)
    print(f"r={r}: Fixed point = {fixed_point:.4f}, f(x*) = {verified:.4f}")

### Exercise 2: Lyapunov Exponent
The Lyapunov exponent measures the rate of separation of nearby trajectories. Implement a function to calculate it.

In [ ]:
def lyapunov_exponent(r, x0=0.1, n=1000):
    """Calculate the Lyapunov exponent for the logistic map."""
    x = x0
    lyap_sum = 0
    
    for i in range(n):
        # Your code here: 
        # The Lyapunov exponent is the average of log|f'(x)|
        # For logistic map: f'(x) = r(1 - 2x)
        
        x = r * x * (1 - x)  # Update x
        
        # Calculate derivative at current x
        derivative = r * (1 - 2 * x)
        
        # Add log of absolute derivative
        if derivative != 0:
            lyap_sum += np.log(abs(derivative))
    
    return lyap_sum / n

# Plot Lyapunov exponent vs r
r_values = np.linspace(2.5, 4.0, 200)
lyap_values = [lyapunov_exponent(r) for r in r_values]

plt.figure(figsize=(10, 6))
plt.plot(r_values, lyap_values, 'b-', linewidth=1)
plt.axhline(y=0, color='r', linestyle='--', alpha=0.5)
plt.xlabel('Growth rate r')
plt.ylabel('Lyapunov Exponent »')
plt.title('Lyapunov Exponent: » > 0 indicates chaos')
plt.grid(True, alpha=0.3)
plt.show()

print("Interpretation:")
print("» < 0: Stable fixed point or periodic orbit")
print("» = 0: Bifurcation point")
print("» > 0: Chaotic dynamics")

## Summary: What We've Learned

1. **Simple rules can create complex behavior**: The logistic map has just one parameter but exhibits incredibly rich dynamics

2. **Period-doubling route to chaos**: Systems often become chaotic through a universal sequence of period-doublings

3. **Sensitivity to initial conditions**: In chaotic systems, tiny differences grow exponentially

4. **Order within chaos**: Even chaotic systems have structure (attractors, Feigenbaum constant)

5. **Practical relevance**: These concepts apply to real-world systems from ecology to economics

### Next Steps
- Try different initial conditions and parameter values
- Explore other 1D maps (tent map, sine map)
- Move on to 2D systems like the Hénon map
- Study continuous systems like the Lorenz attractor (next notebook!)

### Further Reading
- Strogatz, S. H. (2015). *Nonlinear Dynamics and Chaos*
- May, R. M. (1976). "Simple mathematical models with very complicated dynamics"
- Feigenbaum, M. J. (1978). "Quantitative universality for a class of nonlinear transformations"